In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import io
from PIL import Image
from time import sleep
import time
import random
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import WebDriverException
import os
import pandas as pd
import numpy as np


In [2]:
#Crawl basic phone data
# Declare browser
s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

# Khởi tạo trình duyệt Chrome với tùy chọn toàn màn hình
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=s, options=chrome_options)

# Open URL
driver.get("https://www.thegioididong.com/sac-dtdd#c=57&o=14&pi=2")
sleep(random.randint(5,10))

# Scroll down
for i in range(5):
    scroll_script = "window.scrollBy(0, window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1) 

# Scroll up
for i in range(20):
    scroll_script = "window.scrollBy(0, -window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1)
    
# elems2 = driver.find_elements(By.CLASS_NAME , "jBwCF ")
# links = [elem.get_attribute('src') for elem in elems2 if elem.get_attribute('type') == 'product' and elem.get_attribute('src').count("data:image") <=0]

# ================================ GET id
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_57 a')
id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]

# # ================================ GET title
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_57 a')
title = [elem.get_attribute('data-name') for elem in elems if elem.get_attribute('data-name') is not None]

# ================================ GET img_thumb  .item.ajaxed.__cate_42
# elems = driver.find_elements(By.CLASS_NAME, 'thumb')
elems = driver.find_elements(By.CSS_SELECTOR, '.item-img.item-img_57 img')
link_img_thumb = [elem.get_attribute('src') for elem in elems if elem.get_attribute('src') is not None and "Products" in elem.get_attribute('src')]

# ================================ GET href
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_57 a')
href = [elem.get_attribute('href') for i, elem in enumerate(elems) if i % 3 == 0]

# ================================ GET price
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_57 a')
price = [elem.get_attribute('data-price') for elem in elems if elem.get_attribute('data-price') is not None]

# Clean price
price = [int(float(number)) for number in price]

# ================================ GET price pre discount
pre_discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title)+1):
    try:
        discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/span".format(i))
        discount_percent_list.append(discount_percent.text)

        pre_discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/p".format(i))
        pre_discount_list.append(pre_discount.text)

        
        print(i)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))


# ================================ GET brand
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_57 a')
brand = [elem.get_attribute('data-brand') for elem in elems if elem.get_attribute('data-brand') is not None]


# Close browser
driver.close() 

1
No Such Element Exception 2
3
4
5
6
7
8
9
10
11
12
No Such Element Exception 13
14
15
16
17
18
19
20
21
No Such Element Exception 22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
No Such Element Exception 57
58


In [3]:
#Clean discount_list
for i, number in enumerate(pre_discount_list):
    if pre_discount_list[i] and pre_discount_list[i] != '':
        if '.' in pre_discount_list[i]:
            pre_discount_list[i] = int(pre_discount_list[i].replace('.', '').replace('₫', ''))

In [4]:
#Merge & create excel
df1 = pd.DataFrame(list(zip(id_product, title, brand ,href ,link_img_thumb , price)), columns = ['id', 'title' ,'brand', 'href', 'link_img_thumb', 'price'])
# Đặt tên tệp Excel và sheet
excel_file = 'backup_charger.xlsx'
sheet_name = 'Sheet1'

df1['index_']= np.arange(1, len(df1) + 1)

df2 = pd.DataFrame(list(zip(pre_discount_list ,discount_percent_list, discount_idx)), columns = ['pre_discount', 'discount_percent','discount_idx'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

# Lưu DataFrame vào tệp Excel
df3.to_excel(excel_file, sheet_name=sheet_name, index=False, engine='openpyxl')

In [5]:
len(link_img_thumb)

58

In [6]:
df = pd.read_excel('backup_charger.xlsx')

link_img_thumb = df['link_img_thumb'].to_list()
href = df['href'].to_list()
id_product = df['id'].to_list()

print(href)

['https://www.thegioididong.com/sac-dtdd/sac-du-phong-10000mah-magnetic-typecpd-30w-baseus-ppcxm10t-kem-cap-type-c', 'https://www.thegioididong.com/sac-dtdd/sac-du-phong-polymer-10000mah-khong-day-magnetic-type-c-pd-20w-xmobile-t156', 'https://www.thegioididong.com/sac-dtdd/pin-polymer-10000mah-ava-plus-jp208', 'https://www.thegioididong.com/sac-dtdd/pin-polymer-10000mah-ava-ds006', 'https://www.thegioididong.com/sac-dtdd/pin-polymer-10000mah-type-c-ava-pb100s', 'https://www.thegioididong.com/sac-dtdd/pin-polymer-10000mah-type-c-ava-pj-jp260', 'https://www.thegioididong.com/sac-dtdd/pin-sac-du-phong-10000mah-type-c-pd-qc-3-0-30w-anker-a1256', 'https://www.thegioididong.com/sac-dtdd/sac-du-phong-polymer-5000mah-khong-day-magnetic-type-c-pd-20w-xmobile-ds611', 'https://www.thegioididong.com/sac-dtdd/tram-sac-du-phong-di-dong-90000mah-pd140w-220v-450w-baseus-iota-bpe45a', 'https://www.thegioididong.com/sac-dtdd/pin-sac-du-phong-10000mah-type-c-pd-qc3-0-20w-xmobile-y73', 'https://www.thegi

In [7]:
def download_image(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{id_product[i]}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


urls = link_img_thumb

dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
for i, url in enumerate(urls):
	download_image(dowloadPath, url, i)

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
Success  14
Success  15
Success  16
Success  17
Success  18
Success  19
Success  20
Success  21
Success  22
Success  23
Success  24
Success  25
Success  26
Success  27
Success  28
Success  29
Success  30
Success  31
Success  32
Success  33
Success  34
Success  35
Success  36
Success  37
Success  38
Success  39
Success  40
Success  41
Success  42
Success  43
Success  44
Success  45
Success  46
Success  47
Success  48
Success  49
Success  50
Success  51
Success  52
Success  53
Success  54
Success  55
Success  56
Success  57


In [15]:
#Crawl parameters data
list_parameters=[]
for href_items in href:
# for i in range(1):

    # Declare browser
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    # Browser full window
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    
    #Open browser 
    driver.get(href_items)
    sleep(random.randint(1,2))

    # Scroll down
    # for i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 

    elems = driver.find_elements(By.CSS_SELECTOR, '.parameter')
    # id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]
    parameters = [elem.text for elem in elems]

    list_parameters.append(parameters[0])


    #Close browser
    driver.close()

In [ ]:
list_parameters

In [ ]:
efficiency = []
capacity =[]
timefullcharge =[]
input_ =[]
output =[]
core = []
tech = []
size = []
height = []
madein = []
brandof = [] 
brand = []
#Clean parameters data
for i, data in enumerate(list_parameters):
    data = data.replace(":", "")
    data = data.replace('\n', ' ')
    if "Xem thông tin" in data:
        index = data.find(". Xem")
        data = data[0:index]
    if "Jack kết nối" in data:
        index_start = data.find("Jack kết nối")
        index_end = data.find("Dòng sạc")
        data = data[0:index_start-1] + data[index_end:]
    
    start_index = {
        "Hiệu suất sạc": data.find("Hiệu suất sạc"),
        "Dung lượng pin": data.find("Dung lượng pin"),
        "Thời gian sạc đầy pin": data.find("Thời gian sạc đầy pin"),
        "Nguồn vào": data.find("Nguồn vào"),
        "Nguồn ra": data.find("Nguồn ra"),
        "Lõi pin": data.find("Lõi pin"),
        "Công nghệ/Tiện ích": data.find("Công nghệ/Tiện ích"),
        "Kích thước": data.find("Kích thước"),
        "Khối lượng": data.find("Khối lượng"),
        "Thương hiệu của": data.find("Thương hiệu của"),
        "Sản xuất tại": data.find("Sản xuất tại"),
        "Hãng": data.rfind("Hãng")

    }

    # Tìm vị trí của các thông số kết thúc
    end_index = {
        "Hiệu suất sạc": data.find("Dung lượng pin"),
        "Dung lượng pin": data.find("Thời gian sạc đầy pin"),
        "Thời gian sạc đầy pin": data.find("Nguồn vào"),
        "Nguồn vào": data.find("Nguồn ra"),
        "Nguồn ra": data.find("Lõi pin"),
        "Lõi pin": data.find("Công nghệ/Tiện ích"),
        "Công nghệ/Tiện ích": data.find("Kích thước"),
        "Kích thước": data.find("Khối lượng"),
        "Khối lượng": data.find("Thương hiệu của"),
        "Thương hiệu của": data.rfind("Sản xuất tại"),
        "Sản xuất tại": data.find("Hãng"),
        "Hãng": len(data)
    }

    # Khởi tạo một từ điển trống
    chargingadapter_info = {}

    # Duyệt qua các thông số để lấy giá trị
    for key in start_index.keys():
        if start_index[key] != -1 and end_index[key] != -1:
            value = data[start_index[key] + len(key + " "):end_index[key]]
            chargingadapter_info[key] = value
    keys = [
    'Hiệu suất sạc',
    'Dung lượng pin',
    'Thời gian sạc đầy pin',
    'Nguồn vào',
    'Nguồn ra',
    'Lõi pin',
    'Công nghệ/Tiện ích',
    'Kích thước',
    'Khối lượng',
    'Sản xuất tại',
    'Thương hiệu của',
    'Hãng'
    ]

    result_lists = [efficiency, capacity, timefullcharge, input_, output, core, tech, size, height, madein, brandof, brand]

    for index, key in enumerate(keys):
        if key in chargingadapter_info:
            result_lists[index].append(chargingadapter_info[key])
        else:
            result_lists[index].append('Hãng không công bố')
    
    print(chargingadapter_info)


In [30]:
new_df2 = pd.DataFrame(list(zip(efficiency, capacity, timefullcharge, input_, output, core, tech, size, height, madein, brandof, brand)), columns = ['efficiency', 'capacity', 'timefullcharge', 'input_', 'output', 'core', 'tech', 'size', 'height', 'madein', 'brandof', 'brand'])
new_df2.to_excel('backupcharger_info.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [6]:
import requests
import io
from PIL import Image
import os
import threading

def download_image_details(download_path, url, index):
    def download_image_thread():
        try:
            if url.startswith('data:image/'):
                # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
                image_data = url.split(',')[1]
                image_bytes = base64.b64decode(image_data)

                # Tạo một hình ảnh PIL từ các byte và lưu nó
                image = Image.open(io.BytesIO(image_bytes))

                file_path = os.path.join(download_path, f"{index}.jpg")

                with open(file_path, "wb") as f:
                    image.save(f, "JPEG")
            else:
                image_content = requests.get(url, timeout=2).content  # Sử dụng timeout 2 giây
                image_file = io.BytesIO(image_content)
                image = Image.open(image_file)

                file_path = os.path.join(download_path, f"{index}.jpg")

                with open(file_path, "wb") as f:
                    image.save(f, "JPEG")

            print("Success", index)
        except Exception as e:
            print('FAILED -', e)

    thread = threading.Thread(target=download_image_thread)
    thread.start()

# urls = link_img_thumb

# parent_folder = r'D:\CodeUIT\Data\imgs'
# folder_name = 'abc'

# folder_path = os.path.join(parent_folder, folder_name)

# os.makedirs(folder_path, exist_ok=True)

# dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
# for i, url in enumerate(urls):
# 	download_image(dowloadPath, url, i)

In [4]:
df = pd.read_excel('backup_charger.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()
print(id_product)

[316203, 313723, 244687, 245110, 245254, 244689, 315620, 309431, 310238, 308509, 310262, 310261, 314557, 240697, 309433, 307306, 318007, 316970, 316969, 316925, 316152, 315621, 313708, 313707, 312873, 312872, 312792, 310237, 310236, 310234, 310231, 310228, 309716, 309247, 308497, 308491, 291984, 291983, 281379, 253589, 246084, 246083, 245246, 245112, 233280, 229038, 229037, 228892, 227075, 224983, 223828, 145723, 218060, 217431, 214688, 204250, 156075, 146850]


In [7]:
color_item = []
for index, href_items in enumerate(href):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_product[index])

    # elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    # list_color = [elem.text for elem in elems if elem.text is not None]

    # len_color = len(elems)
    # len_loop = len_color+2
    
    elems_loop = driver.find_elements(By.CSS_SELECTOR, '.scrolling_inner .box01__tab.scrolling .item.itemTab')
    loop = [elem_loop for elem_loop in elems_loop]
    list_color = []

    for i in range(1,len(loop)):
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if check[0] != 'color-images-gallery':
            if check[0] == 'specification-gallery':
                break
            else: continue

        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]
        list_color.append(color[0])

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None and 'png' not in elem.get_attribute('data-src')]
        

        for item in href_color:
            if item is None or str(id_product[index]) not in item:
                href_color.remove(item)

        urls = href_color

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_product[index])
        folder_path = os.path.join(parent_folder, folder_name, color[0])

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path
        
        thumb_color[0] = 'https:' + thumb_color[0]
        download_image_details(dowloadPath, thumb_color[0], 'thumb')

        for z, url in enumerate(urls):
            url = 'https:' + url
            download_image_details(dowloadPath, url, z)
    if len(list_color) >1 :
        color_item.append(', '.join(list_color[0:len(list_color)-1]))
    else: color_item.append(list_color[0])
    
    driver.close()

316203
Success thumb
Success 1
Success 2
Success 3
Success 0
Success 4
Success thumb
Success 1
Success 3
Success 2
Success 0
Success 5
Success 4
Success thumb
Success 0
Success 4
Success 1
Success 3
Success 2
Success 5
Success 6
Success thumb
Success 1
Success 0
Success 2
Success 4
Success 5
Success 6
Success 3
313723
Success thumb
Success 2
Success 1
Success 0
Success 3
Success 4
Success 10
Success 15
Success 12
Success 9
Success 11
Success 13
Success 14
Success 8
Success 6
Success 7
Success 5
Success thumb
Success 3
Success 2
Success 0
Success 1
Success 6
Success 7
Success 10
Success 13
Success 8
Success 9
Success 4
Success 14
Success 5
Success 11
Success 12
Success thumb
Success 1
Success 0
Success 2
Success 4
Success 7
Success 3
Success 11
Success 6
Success 5
Success 8
Success 9
Success 12
Success 14
Success 10
Success 15
Success 13
Success 16
Success thumb
FAILED - HTTPSConnectionPool(host='cdn.tgdd.vn', port=443): Read timed out.
FAILED - HTTPSConnectionPool(host='cdn.tgdd.vn', p

Success 1
Success 3


In [9]:
len(color_item)

58

In [10]:
new_df2 = pd.DataFrame(list(zip(color_item)), columns = ['color_item'])
new_df2.to_excel('color_item_backupcharger.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')